<h2 align='center'>PROJECT 4 - ACCRE</h2>

#### Using just the fullsample.csv jobs dataset:

### 1. Calculate some descriptive statistics for how many jobs per hour are being completed. What does the completions per hour look like over the time span of the dataset? Are there weekly trends, and has it been increasing over the last year?

In [1]:
from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
full_sample = pd.read_csv('../data/fullsample.csv',
                          usecols = [0,1,2,3,
                                     6,7,
                                     10,11])

In [8]:
full_sample.info()
display(full_sample.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7395885 entries, 0 to 7395884
Data columns (total 8 columns):
 #   Column     Dtype         
---  ------     -----         
 0   JOBID      object        
 1   STATE      object        
 2   BEGIN      datetime64[ns]
 3   END        datetime64[ns]
 4   REQTIME    object        
 5   USEDTIME   object        
 6   PARTITION  object        
 7   EXITCODE   object        
dtypes: datetime64[ns](2), object(6)
memory usage: 451.4+ MB


,JOBID,STATE,BEGIN,END,REQTIME,USEDTIME,PARTITION,EXITCODE
0,30616928,RUNNING,2021-07-31 22:15:00,1901-01-01 11:59:59,10:04:00,67-22:14:22,production,0:0
1,30853133,COMPLETED,2021-08-06 11:36:09,2021-09-05 11:36:32,30-00:00:00,30-00:00:23,cgw-platypus,0:0
2,30858137,COMPLETED,2021-08-06 19:04:39,2021-09-05 19:04:53,30-00:00:00,30-00:00:14,cgw-tbi01,0:0
3,30935078,COMPLETED,2021-08-09 16:52:51,2021-09-07 20:52:55,29-04:00:00,29-04:00:04,cgw-platypus,0:0
4,31364111_2,COMPLETED,2021-08-17 07:45:07,2021-09-10 16:45:24,24-09:00:00,24-09:00:17,production,0:0


In [4]:
# full_sample[(full_sample['BEGIN']=='Unknown')].count()

full_sample['BEGIN'] = full_sample['BEGIN']\
                                           .str.replace('Unknown', 
                                                        '1901-01-01T11:59:59')

In [5]:
# full_sample[(full_sample['END']=='Unknown')].count()

full_sample['END'] = full_sample['END']\
                                       .str.replace('Unknown', 
                                                    '1901-01-01T11:59:59')

In [6]:
full_sample['BEGIN'] = pd.to_datetime(full_sample['BEGIN'])

In [7]:
full_sample['END'] = pd.to_datetime(full_sample['END'])

In [10]:
full_sample = full_sample.sort_values('END')

In [13]:
# MOVING COUNT OF JOBS PER HOUR -- 

full_sample['jobs_per_hour'] = full_sample.rolling('1h', 
                                                   on = 'END')['JOBID'].count()
display(full_sample[(full_sample['END']!='1901-01-01 11:59:59')].head())

,JOBID,STATE,BEGIN,END,REQTIME,USEDTIME,PARTITION,EXITCODE,jobs_per_hour
6640577,24460577,COMPLETED,2020-10-01 00:05:55,2020-10-01 00:10:15,12:00:00,00:04:20,production,0:0,1.0
6640635,24460647,COMPLETED,2020-10-01 00:10:38,2020-10-01 00:12:58,2-00:00:00,00:02:20,sam,0:0,2.0
6640627,24460639,COMPLETED,2020-10-01 00:11:22,2020-10-01 00:13:31,00:30:00,00:02:09,production,0:0,3.0
6640651,24460668,COMPLETED,2020-10-01 00:15:16,2020-10-01 00:17:23,2-00:00:00,00:02:07,sam,0:0,4.0
6640654,24460671,COMPLETED,2020-10-01 00:17:08,2020-10-01 00:17:53,00:30:00,00:00:45,production,0:0,5.0


In [26]:
# GROUP BY YEAR-MONTH-DAY-HOUR?

jobs_by_day_hour = (full_sample
                               .groupby('END'.dt.date)
                               .rolling('1h', 
                                        on = 'END')['JOBID']
                               .count()
                               .reset_index()
                               .set_index('END')
                   )

AttributeError: 'str' object has no attribute 'dt'

In [ ]:
# full_sample['DAYS_USED'] = full_sample['USEDTIME'].str.split('-', expand=True)[0]

In [ ]:
# full_sample['HOURS_USED'] = full_sample['USEDTIME'].str.split('-', expand=True)[1]

In [ ]:
# IF CREATING SMALLER TABLE TO TEST CODE
#
# out_mem_test = full_sample[(full_sample['STATE']=='OUT_OF_MEMORY')]
# display(out_mem_test.head())

In [ ]:
job_state_counts = full_sample\
                               .groupby('STATE')['JOBID']\
                               .count()\
                               .sort_values(ascending=False)
                    #          .describe()
job_state_counts

In [ ]:
# full_sample['STATE'].unique()

In [ ]:
# zero_exits = full_sample[(full_sample['STATE']!='COMPLETED') & (full_sample['EXITCODE'] == '0:0')]
# print(zero_exits['STATE'].unique())

#### Using just the fullsample.csv jobs dataset:

### 2. Does the job state affect completions per hour? i.e. if I only look at jobs with exit code 0:0 in the "COMPLETED" state, is that a similar number of completions per hour as with all jobs, failed or cancelled? This will indicate if the load on the scheduler is by user design or is a result of users not sufficiently testing their jobs before submitting very large arrays. We also expect that most job completions will be in the "production" partition, but is this actually true?

In [ ]:
'''
EXITCODE 0:0            = SUCCESS
         0:15, 0:9, 0:2 = CANCELLED
         1:0, 2.0       = FAILED
'''
exit_code_counts = full_sample\
                               .groupby('EXITCODE')['JOBID']\
                               .count()\
                               .sort_values(ascending=False)
exit_code_counts

In [25]:
# MOVING COUNT GROUPING BY EXIT CODE -- 

group_by_exit_code = (full_sample
                                 .groupby('EXITCODE')
                                 .rolling('1h', 
                                          on = 'END')['JOBID']
                                 .count()
                                 .reset_index()
                                 .set_index('END')
                      )
group_by_exit_code

KeyError: "None of ['END'] are in the columns"

#### Using the two log files:

### 3.	Calculate some descriptive statistics about how often the scheduler was unresponsive, how long these periods of time were, and create a time series plot of when the scheduler was having difficulties.

In [14]:
ce5_raw = pd.read_table("../data/slurm_wrapper_ce5.log", 
                        engine='python',
                        sep=" - ", 
                       # skiprows=[0], 
                       # header=0,
                        names=['ce5_DateTime', 
                               'ce5_User', 
                               'ce5_Retry',
                               'ce5_Time_Spent',
                               'ce5_Return_Code',
                               'ce5_Command']
                       ) 
                        # parse_dates=[1]).rename(columns=lambda x: x.strip())

In [17]:
ce5_raw.info()
display(ce5_raw.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4770893 entries, 0 to 4770892
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   DateTime     datetime64[ns]
 1   User         object        
 2   Retry        object        
 3   Time_Spent   object        
 4   Return_Code  object        
 5   Command      object        
dtypes: datetime64[ns](1), object(5)
memory usage: 218.4+ MB


,DateTime,User,Retry,Time_Spent,Return_Code,Command
0,2020-10-16 08:15:39.278699,user 0,retry 0,time 0.07347559928894043,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
1,2020-10-16 08:18:08.313309,user 0,retry 0,time 0.18363237380981445,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
2,2020-10-16 08:22:48.128689,user 0,retry 0,time 0.07547116279602051,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
3,2020-10-16 08:25:13.257408,user 0,retry 0,time 0.09484362602233887,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."
4,2020-10-16 08:31:01.460723,user 0,retry 0,time 0.07498788833618164,returncode 0,"command ['/usr/bin/sacct', '-u', 'appelte1', '..."


In [16]:
ce5_raw['DateTime'] = pd.to_datetime(ce5_raw.DateTime)

In [19]:
ce5_hrs_range = ce5_raw['DateTime'].max() - ce5_raw['DateTime'].min()
display(ce5_hrs_range)

Timedelta('356 days 13:43:38.414759')

In [20]:
ce6_raw = pd.read_table("../data/slurm_wrapper_ce6.log", 
                        engine='python',
                        sep=" - ", 
                       # skiprows=[0], 
                       # header=0,
                        names=['ce6_DateTime', 
                               'ce6_User', 
                               'ce6_Retry',
                               'ce6_Time_Spent',
                               'ce6_Return_Code',
                               'ce6_Command']
                       ) 
                        # parse_dates=[1]).rename(columns=lambda x: x.strip())

In [21]:
ce6_raw.info()
display(ce6_raw.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4776520 entries, 0 to 4776519
Data columns (total 6 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   DateTime     object
 1   User         object
 2   Retry        object
 3   Time_Spent   object
 4   Return_Code  object
 5   Command      object
dtypes: object(6)
memory usage: 218.7+ MB


,DateTime,User,Retry,Time_Spent,Return_Code,Command
0,2020-10-16 10:37:44.163454,user 9202,retry 0,time 0.08495402336120605,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
1,2020-10-16 10:37:44.206654,user 9202,retry 0,time 0.08943057060241699,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
2,2020-10-16 10:37:44.218760,user 9202,retry 0,time 0.05928945541381836,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
3,2020-10-16 10:37:44.256403,user 9202,retry 0,time 0.038695573806762695,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."
4,2020-10-16 10:37:44.611603,user 9202,retry 0,time 0.03343677520751953,returncode 0,"command ['/usr/bin/scontrol', 'show', 'job', '..."


In [22]:
ce6_raw['DateTime'] = pd.to_datetime(ce6_raw.DateTime)

In [23]:
ce6_hrs_range = ce6_raw['DateTime'].max() - ce6_raw['DateTime'].min()
display(ce6_hrs_range)

Timedelta('356 days 11:22:36.756104')

### 4.	Finally, combine the time series information from the two datasets together to see how well correlated heavy job-completion load is with the unresponsiveness, and to see if there is some threshold of job completions per hour that generally results in unresponsiveness.